In [19]:

%pip install pandas
%pip install openpyxl

import pandas as pd


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [20]:
# Definerer filbanen for den opplastede Excel-filen
file_path_import = '/Users/torbjornsvendsen/Downloads/Case_Restaurant_eng.xlsx' # Denne er dynamisk og kan endres etter hvor filen befinner seg

# Definerer filbanen for ferdig bearbeidet fil til PowerBI
file_path_export = '/Users/torbjornsvendsen/Downloads/Case_Restaurant_eng_preppet311024.xlsx' # Denne er dynamisk og kan endres etter hvor filen befinner seg

In [21]:
# Importerer hvert ark til en DataFrame
dim_employee = pd.read_excel(
    file_path_import, 
    sheet_name='dimEmployee')
dim_product = pd.read_excel(
    file_path_import, 
    sheet_name='dimProduct')
sales = pd.read_excel(
    file_path_import, 
    sheet_name='Sales')
timesheet = pd.read_excel(
    file_path_import, 
    sheet_name='Time sheet')

In [22]:
# Viser data for å bekrefte innlasting
dim_employee.head(5), 
dim_product.head(5), 
sales.head(5), 
timesheet.head(5)

,Date,POS,EmployeeId,From,To
0,2023-08-08,Sandnes,1172190,2023-08-08 15:00:00,2023-08-08 23:00:00
1,2023-08-08,Sandnes,5622025,2023-08-08 15:15:00,2023-08-08 23:15:00
2,2023-08-09,Sandnes,5625995,2023-08-09 07:30:00,2023-08-09 15:30:00
3,2023-08-11,Sandnes,5625995,2023-08-11 07:30:00,2023-08-11 15:30:00
4,2023-08-11,Sandnes,5667430,2023-08-11 08:00:00,2023-08-11 16:00:00


In [23]:
# Steg 2: Databearbeiding og relasjonskoblinger mellom tabeller

# Slår sammen dimEmployee med TimeSheet på EmployeeId
timesheet = timesheet.merge(
    dim_employee, 
    on='EmployeeId', 
    how='left')

# Slår sammen dimProduct med Sales på ProductId
sales = sales.merge(
    dim_product, 
    on='ProductId', 
    how='left')

In [24]:
# Viser data for å bekrefte relasjonskoblinger
timesheet.head(5), 
sales.head(5)

,Order number,POS,ProductId,Number of orders,Sales amount exluding VAT,Discount,Cost of goods,Date,Closest hour,Product name
0,341320944,Sandnes,1276605,1,630.43,0.0,350.0,2023-08-11,22,Classic Burger
1,343522530,Sandnes,1276605,2,1260.86,0.0,700.0,2023-08-18,14,Classic Burger
2,341784333,Sandnes,1276605,1,630.43,0.0,350.0,2023-08-22,15,Classic Burger
3,340922721,Sandnes,1519623,1,543.48,0.0,350.0,2023-08-24,19,Spicy Chicken Sandwich
4,343522533,Sandnes,1398942,1,508.70,0.0,350.0,2023-08-25,18,Vegetarian Pizza


In [25]:
# Oppretter nøkkelmålinger og variabler for videre analyser i Power BI
# Totalsalg: Sum av "Sales amount excluding VAT" i Sales tabellen
sales[
    'TotalSales'
    ] = sales[
        'Sales amount exluding VAT'].sum() #Summererer

# Salg per time: Summerer "Sales amount excluding VAT" per nærmeste time og lokasjon (POS)
sales_per_hour = sales.groupby(
    ['Closest hour', 'POS']
    )['Sales amount exluding VAT'].sum().reset_index(
        name='SalesAmountPerHour')

# Antall ansatte på vakt per time: Teller antall unike EmployeeId per time og lokasjon i TimeSheet
timesheet[
    'Hour'] = pd.to_datetime(
        timesheet[
            'From']).dt.hour

employees_per_hour = timesheet.groupby(
    ['Hour', 'POS']
    )['EmployeeId'].nunique(
    ).reset_index(
        name='EmployeesOnShiftPerHour')

# Margin per produkt: Beregner marginen for hvert produkt som differansen mellom "Sales amount excluding VAT" og "Cost of goods"
sales[
    'ProductMargin'] = sales[
        'Sales amount exluding VAT'] - sales['Cost of goods']

# Gjennomsnittsmargin per ordre
sales[
    'AverageMarginPerOrder'] = sales[
        'ProductMargin'].mean()

# Salgsmengde: Sum av "Number of Orders" per produkt
sales_quantity = sales.groupby(
    'ProductId')[
        'Number of orders'].sum(
        ).reset_index(
            name='SalesQuantity')


In [26]:
# Viser bearbeidet data
sales.head(5), 
timesheet.head(5), 
sales_per_hour.head(5), 
employees_per_hour.head(5), 
sales_quantity.head(5)

,ProductId,SalesQuantity
0,1086687,290
1,1086693,418
2,1086699,387
3,1086705,65
4,1086711,1023


In [27]:
# Eksporterer data
with pd.ExcelWriter(
        file_path_export, 
        engine='openpyxl'
        ) as writer: # Eksporter hver DataFrame til et eget ark i samme .xlsx-fil
    sales.to_excel(
        writer, 
        sheet_name='Sales', 
        index=False)  # Eksporterer 'sales' DataFrame til ark kalt 'Sales' uten indeks
    sales_per_hour.to_excel(
        writer, 
        sheet_name='Sales_Per_Hour', 
        index=False)  # Eksporterer 'sales_per_hour' til ark 'Sales_Per_Hour'
    employees_per_hour.to_excel(
        writer, 
        sheet_name='Employees_Per_Hour', 
        index=False)  # Eksporterer 'employees_per_hour' til 'Employees_Per_Hour'
    sales_quantity.to_excel(
        writer, 
        sheet_name='Sales_Quantity', 
        index=False)  # Eksporterer 'sales_quantity' til ark 'Sales_Quantity'

print(
    "Filen befinner seg nå i filbanen:", 
    file_path_export)

Filen befinner seg nå i filbanen: /Users/torbjornsvendsen/Downloads/Case_Restaurant_eng_preppet311024.xlsx
